In [27]:
#!/usr/bin/env python3

import pandas as pd
import errno    
import os
import numpy as np
import math

pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

'''
Author: Enrico Ceccolini
    TODO write the description
'''

'\nAuthor: Enrico Ceccolini\n    TODO write the description\n'

In [28]:
datadir = "/datasets/eurora_data/db/"
infile_jobs_to_nodes = datadir + "job_nodes.csv"
infile_jobs = datadir + "jobs.csv"

# settings
interval_comment = "WholeData"
suffix = "_1min_"

start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2015-08-11')


## notice that 43 doesn't exists in the db
# cpus=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']
cpus=['01'] # test

### Read job2nodes data

In [29]:
jobs_to_nodes_whole_data = pd.read_csv(infile_jobs_to_nodes)
print("jobs_to_nodes_whole_data contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data contains 469095 records


#### solve the problem of jobs runned on node 129

In [30]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data[jobs_to_nodes_whole_data['node_id'] != 129]
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 469020 records


#### solve the problem of duplicates job_id_string, node_id
this is probably caused by the concurrency writing of the database 

In [31]:
jobs_to_nodes_whole_data = jobs_to_nodes_whole_data.drop_duplicates(subset=['job_id_string', 'node_id'])
print("jobs_to_nodes_whole_data now contains {} records".format(jobs_to_nodes_whole_data.shape[0]))

jobs_to_nodes_whole_data now contains 444610 records


### Read jobs data

In [32]:
jobs_whole_data = pd.read_csv(infile_jobs)
print("jobs_whole_data contains {} records".format(jobs_whole_data.shape[0]))

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


jobs_whole_data contains 405771 records


delete the jobs with missing end time for the moment

In [33]:
completed_jobs_data = jobs_whole_data[jobs_whole_data.end_time != '0000-00-00 00:00:00']
print("completed_jobs_data contains {} records".format(completed_jobs_data.shape[0]))

completed_jobs_data contains 404871 records


In [34]:
completed_jobs_data = completed_jobs_data.dropna(subset=['run_start_time', 'end_time'])
print("completed_jobs_data contains {} records".format(completed_jobs_data.shape[0]))

completed_jobs_data contains 404871 records


### Drop jobs with running time less than 1 min

In [35]:
large_jobs = completed_jobs_data[pd.to_datetime(completed_jobs_data['end_time']) - pd.to_datetime(completed_jobs_data['run_start_time']) > np.timedelta64(1, 'm')]
print("large jobs (running time > 1 min) contains {} jobs".format(large_jobs.shape[0]))

large jobs (running time > 1 min) contains 71457 jobs


In [36]:
#ids = (jobs_to_nodes_whole_data['job_id_string'] )
#node_data.loc[ids, ['active_cores', 'active_jobs', 'active_gpus', 'active_mics']] = [+job_cores, +1, +job_gpus, +job_mics]

#### merge

In [37]:
merged_jobs_to_nodes = pd.merge(jobs_to_nodes_whole_data, large_jobs, on='job_id_string')
print(merged_jobs_to_nodes.shape[0])

97760


In [38]:
df = merged_jobs_to_nodes.groupby(['job_id_string']).size().reset_index(name='counts')
df = df.sort_values('counts')
df = df[df['counts'] > 1]
print("{} jobs runned on more than one nodes".format(df.shape[0]))
df

9467 jobs runned on more than one nodes


,job_id_string,counts
4609,1200342.node129,2
4284,1199795.node129,2
1665,1009590.node129,2
66891,922445.node129,2
66487,920767.node129,2
3823,1199142.node129,2
3196,1017498.node129,2
3195,1017497.node129,2
1666,1009592.node129,2
64271,815910.node129,2


#### drop the unused columns

In [39]:
large_jobs.head(1)

,job_id,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,time_req,deleted,dependency,exit_status,pbs_exit_code
0,1,498458.node129,rth2e6t0,parallel,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,alupi000@node129.eurora.cineca.it,1,16,14,04:00,NaN,NaN,COMPLETED,NaN


In [41]:
large_jobs = large_jobs.drop(['job_id', 'job_name', 'queue', 'user', 'deleted', 'dependency', 'pbs_exit_code'], axis=1)
large_jobs.head(1)

KeyError: "['job_id' 'job_name' 'queue' 'user' 'deleted' 'dependency' 'pbs_exit_code'] not found in axis"

In [42]:
# drop the jobs out of time interval
jobs_data_interval = large_jobs.loc[(pd.to_datetime(large_jobs['run_start_time']) >= start_time) & (pd.to_datetime(large_jobs['end_time']) <= end_time)]
print("There are {} jobs executed in the interval".format(jobs_data_interval.shape[0]))

There are 71454 jobs executed in the interval


In [43]:
### I will probably create a new dataframe "job_id_string: real_power"
# add two new column: real_power and entire_node
# completed_jobs_data_interval['entire_node'] = pd.Series()
# completed_jobs_data_interval['real_power'] = pd.Series(0)
# completed_jobs_data_interval.head(1)

### compute the real power consumption

In [44]:
### debug
#print(((pd.to_datetime('2014-03-31 00:36:56')) - pd.to_datetime('2014-03-31 00:36:51')))
#print(math.ceil(((pd.to_datetime('2014-03-31 00:36:56')) - pd.to_datetime('2014-03-31 00:36:51'))/ np.timedelta64(1, 'm')))
#print(((pd.to_datetime(job['end_time']) + np.timedelta64(1, 'm')) - pd.to_datetime(job['run_start_time'])))
#print(((pd.to_datetime(job['end_time']) + np.timedelta64(1, 'm')) - pd.to_datetime(job['run_start_time'])) / np.timedelta64(1, 'm'))
#print(math.ceil(((pd.to_datetime(job['end_time'])) - pd.to_datetime(job['run_start_time'])) / np.timedelta64(1, 'm')))

In [45]:
### add the real_pow column to the dataset
num_jobs = large_jobs.shape[0]
real_pow = pd.Series(0, index=np.arange(num_jobs))
print("there are {} - large_jobs".format(num_jobs))
print("there are {} - merged_jobs_to_nodes".format(merged_jobs_to_nodes.shape[0]))

there are 71457 - large_jobs
there are 97760 - merged_jobs_to_nodes


In [46]:
large_jobs.index[large_jobs.shape[0]-1]

405769

In [97]:
i = 0
for index, row in large_jobs.iterrows():
    print(i)
    print()
    real_pow[i] = calculate_job_consumption(row['job_id_string'])
    i = i + 1
    if(i > 10):
        break

0

job 498458.node129 use 16 cores in this node
job 498458.node129 start at 2014-03-31 00:30:02 and end at 2014-03-31 01:22:40
job 498458.node129 requires 1 nodes and 16 cores
[17]
node 17
interval_measurements 0
51/51 intervals missing
Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 0.1, timestamp, pow_tot_0, pow_tot_1, active_cores, active_jobs, active_gpus, active_mics]
Index: []
Empty DataFrame
Columns: [index, Unnamed: 0, Unnamed: 0.1, pow_tot_0, pow_tot_1]
Index: []
partial measurement: 0
0/1 node measurements are good
1

job 498459.node129 use 16 cores in this node
job 498459.node129 start at 2014-03-31 00:36:51 and end at 2014-03-31 00:40:13
job 498459.node129 requires 1 nodes and 16 cores
[9]
node 09
interval_measurements 3
0/3 intervals missing
    Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
37        9281          9281  2014-03-31 00:37:00   97.674106   99.556241            16            1         

interval_measurements 240
0/240 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
55         9299          9299  2014-03-31 00:55:00   96.726417   99.046400            16            1            0            0
56         9300          9300  2014-03-31 00:56:00  100.536774  102.052164            16            1            0            0
57         9301          9301  2014-03-31 00:57:00   98.028684  101.326939            16            1            0            0
58         9302          9302  2014-03-31 00:58:00  100.868140  102.100652            16            1            0            0
59         9303          9303  2014-03-31 00:59:00   99.348847  100.765943            16            1            0            0
60         9304          9304  2014-03-31 01:00:00   98.757263  101.124788            16            1            0            0
61         9305          9305  2014-03-31 01:01:00   9

interval_measurements 157
83/240 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
55         9299          9299  2014-03-31 00:55:00   96.839566   99.036541            16            1            0            0
56         9300          9300  2014-03-31 00:56:00   99.907070  102.268786            16            1            0            0
57         9301          9301  2014-03-31 00:57:00   95.795042   98.258969            16            1            0            0
58         9302          9302  2014-03-31 00:58:00   96.655965   99.937514            16            1            0            0
59         9303          9303  2014-03-31 00:59:00   96.482056   99.043429            16            1            0            0
60         9304          9304  2014-03-31 01:00:00   97.379960   99.203653            16            1            0            0
61         9305          9305  2014-03-31 01:01:00   

interval_measurements 240
0/240 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
55         9299          9299  2014-03-31 00:55:00   93.304878   99.100871            16            1            0            0
56         9300          9300  2014-03-31 00:56:00   98.268354  103.306720            16            1            0            0
57         9301          9301  2014-03-31 00:57:00   95.271658   99.493978            16            1            0            0
58         9302          9302  2014-03-31 00:58:00   97.897816  101.379820            16            1            0            0
59         9303          9303  2014-03-31 00:59:00   96.705010  101.127144            16            1            0            0
60         9304          9304  2014-03-31 01:00:00   96.395435  100.781903            16            1            0            0
61         9305          9305  2014-03-31 01:01:00   9

interval_measurements 240
0/240 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
55         9299          9299  2014-03-31 00:55:00   97.070661  104.590760            16            1            0            0
56         9300          9300  2014-03-31 00:56:00   98.824344  105.263515            16            1            0            0
57         9301          9301  2014-03-31 00:57:00   94.999445  102.731730            16            1            0            0
58         9302          9302  2014-03-31 00:58:00   97.643777  104.860229            16            1            0            0
59         9303          9303  2014-03-31 00:59:00   95.964866  104.343797            16            1            0            0
60         9304          9304  2014-03-31 01:00:00   96.863126  104.202752            16            1            0            0
61         9305          9305  2014-03-31 01:01:00   9

...

interval_measurements 55
0/55 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
264        9508          9508  2014-03-31 04:24:00   23.248012   24.099386            16            1            0            0
265        9509          9509  2014-03-31 04:25:00   23.140140   23.967555            16            1            0            0
266        9510          9510  2014-03-31 04:26:00   23.060528   23.868860            16            1            0            0
267        9511          9511  2014-03-31 04:27:00   23.002579   23.811903            16            1            0            0
268        9512          9512  2014-03-31 04:28:00   23.437651   24.232350            16            1            0            0
269        9513          9513  2014-03-31 04:29:00   23.645416   24.497100            16            1            0            0
270        9514          9514  2014-03-31 04:30:00   24.

interval_measurements 55
0/55 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
264        9508          9508  2014-03-31 04:24:00   24.749669   24.152626            16            1            0            0
265        9509          9509  2014-03-31 04:25:00   24.975907   24.310959            16            1            0            0
266        9510          9510  2014-03-31 04:26:00   24.972956   24.340047            16            1            0            0
267        9511          9511  2014-03-31 04:27:00   25.119086   24.432360            16            1            0            0
268        9512          9512  2014-03-31 04:28:00   24.938470   24.344342            16            1            0            0
269        9513          9513  2014-03-31 04:29:00   25.170074   24.440122            16            1            0            0
270        9514          9514  2014-03-31 04:30:00   25.

interval_measurements 55
0/55 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
264        9508          9508  2014-03-31 04:24:00   24.416017   23.989065            16            1            0            0
265        9509          9509  2014-03-31 04:25:00   24.551783   24.033170            16            1            0            0
266        9510          9510  2014-03-31 04:26:00   24.550937   24.012044            16            1            0            0
267        9511          9511  2014-03-31 04:27:00   24.325559   23.962323            16            1            0            0
268        9512          9512  2014-03-31 04:28:00   24.235510   23.914337            16            1            0            0
269        9513          9513  2014-03-31 04:29:00   24.424679   24.292698            16            1            0            0
270        9514          9514  2014-03-31 04:30:00   24.

interval_measurements 0
55/55 intervals missing
Empty DataFrame
Columns: [Unnamed: 0, Unnamed: 0.1, timestamp, pow_tot_0, pow_tot_1, active_cores, active_jobs, active_gpus, active_mics]
Index: []
Empty DataFrame
Columns: [index, Unnamed: 0, Unnamed: 0.1, pow_tot_0, pow_tot_1]
Index: []
partial measurement: 0
node 63
interval_measurements 55
0/55 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
264        9508          9508  2014-03-31 04:24:00   23.970941   25.405630            16            1            0            0
265        9509          9509  2014-03-31 04:25:00   23.953386   25.338259            16            1            0            0
266        9510          9510  2014-03-31 04:26:00   23.825619   25.184734            16            1            0            0
267        9511          9511  2014-03-31 04:27:00   23.987392   25.245188            16            1            0        

interval_measurements 200
0/200 intervals missing
     Unnamed: 0  Unnamed: 0.1            timestamp   pow_tot_0   pow_tot_1  active_cores  active_jobs  active_gpus  active_mics
264        9508          9508  2014-03-31 04:24:00   52.122945   50.974950            12            5            0            0
265        9509          9509  2014-03-31 04:25:00   28.847453   30.232419            12            5            0            0
266        9510          9510  2014-03-31 04:26:00   28.338809   30.147452            12            5            0            0
267        9511          9511  2014-03-31 04:27:00   27.131971   27.585449            12            5            0            0
268        9512          9512  2014-03-31 04:28:00   27.494302   28.014798            12            5            0            0
269        9513          9513  2014-03-31 04:29:00   27.337038   27.763436            12            5            0            0
270        9514          9514  2014-03-31 04:30:00   2

In [98]:
real_pow

0           0
1         109
2         616
3         102
4           0
5           0
6           0
7          10
8        1408
9         613
10         35
11          0
12          0
13          0
14          0
15          0
16          0
17          0
18          0
19          0
20          0
21          0
22          0
23          0
24          0
25          0
26          0
27          0
28          0
29          0
30          0
31          0
32          0
33          0
34          0
35          0
36          0
37          0
38          0
39          0
40          0
41          0
42          0
43          0
44          0
45          0
46          0
47          0
48          0
49          0
50          0
51          0
52          0
53          0
54          0
55          0
56          0
57          0
58          0
59          0
60          0
61          0
62          0
63          0
64          0
65          0
66          0
67          0
68          0
69          0
70          0
71    

In [78]:
large_jobs

,job_id,job_id_string,start_time,run_start_time,end_time,node_req,cpu_req,mem_req,time_req,exit_status
0,1,498458.node129,2014-03-31 00:30:02,2014-03-31 00:30:02,2014-03-31 01:22:40,1,16,14,04:00,COMPLETED
1,2,498459.node129,2014-03-31 00:36:51,2014-03-31 00:36:51,2014-03-31 00:40:13,1,16,14,01:00,COMPLETED
2,3,498460.node129,2014-03-31 00:54:37,2014-03-31 00:54:38,2014-03-31 04:55:17,6,96,84,04:00,COMPLETED
3,4,498461.node129,2014-03-31 01:00:01,2014-03-31 01:00:02,2014-03-31 05:01:18,1,16,14,04:00,COMPLETED
4,5,498462.node129,2014-03-31 01:30:02,2014-03-31 01:30:02,2014-03-31 02:24:29,1,16,14,04:00,COMPLETED
5,6,498463.node129,2014-03-31 02:30:02,2014-03-31 02:30:02,2014-03-31 03:24:28,1,16,14,04:00,COMPLETED
6,7,498464.node129,2014-03-31 03:30:09,2014-03-31 03:30:09,2014-03-31 04:24:51,1,16,14,04:00,COMPLETED
7,8,498465.node129,2014-03-31 04:00:17,2014-03-31 04:00:17,2014-03-31 07:50:11,1,1,2,06:00,COMPLETED
8,9,498466.node129,2014-03-31 04:00:17,2014-03-31 04:00:18,2014-03-31 04:23:30,11,176,154,06:00,COMPLETED
9,10,498467.node129,2014-03-31 04:23:23,2014-03-31 04:23:30,2014-03-31 05:19:43,11,176,154,06:00,COMPLETED


In [68]:
#### calculate the power consumption of a specifified job



job_id_string = "498462.node129" # non ha misurazioni
job_id_string = "498463.node129" # non ha misurazioni
job_id_string = "498464.node129" # non ha misurazioni
job_id_string = "498465.node129" # [ 35]

job_id_string = "498699.node129" # [ 37]
job_id_string = "498458.node129" # non ha misurazioni
job_id_string = "498458.node129" # non ha misurazioni
job_id_string = "498461.node129" # single node 16 cores [ 27]
job_id_string = "500752.node129" # 2 nodi, 24 cores [ 16 20] 20 totalmente assente
job_id_string = "500899.node129" # [ 15 23 32 28 25 19]

job_id_string = "498467.node129"
job_id_string = "498459.node129" # single node 16 cores [ 9]
job_id_string = "498460.node129" # 6 nodi, 96 cores [ 9 28 25 19 26 29]
job_id_string = "498466.node129" # [ 36 38 40 42 50 56 59 61 62 63 54]
job_id_string = "500752.node129" # 2 nodi, 24 cores [ 16 20] 20 totalmente assente


print("job_id_string: {}".format(job_id_string))



job_id_string: 500752.node129


In [69]:
### TODO I probably have to load more than one node measurements file
def calculate_job_consumption(job_id_string):
    
    job_real_pow_consumption = 0
    job_runned_alone = True
    good_nodes = 0 # node where we have at most 1 measurement in the interval
    partial_pow_consumption = 0
    
    job_to_nodes = merged_jobs_to_nodes[merged_jobs_to_nodes['job_id_string'] == job_id_string]
    job_row = job_to_nodes.iloc[0]
    job_to_nodes_unique = job_to_nodes.node_id.unique() # series with the nodes id
    ### print some jobs info
    print("job {} use {} cores in this node".format(job_id_string, str(job_row['ncpus'])))
    print("job {} start at {} and end at {}".format(job_id_string, str(job_row['run_start_time']), str(job_row['end_time'])))
    print("job {} requires {} nodes and {} cores".format(job_id_string, str(job_row['node_req']), str(job_row['cpu_req'])))
    print(job_to_nodes_unique)
    
    
    for index, row in job_to_nodes.iterrows():
        ### add a 0 to the number if is less than 10 ("1" -> "01", ...)
        node_str = "0" + str(row['node_id']) if row['node_id'] < 10 else str(row['node_id'])
        print("node {}".format(node_str))
        
        ### open the measurements node file I/O
        infile_node = datadir + "CPUs/" + interval_comment + "/node" + node_str + suffix + interval_comment + "_active_cores_and_jobs.csv"
        node_measurements = pd.read_csv(infile_node)
    
        ### take only the entries of the interval where the job was running
        interval_measurements = node_measurements.loc[(pd.to_datetime(node_measurements['timestamp']) >= pd.to_datetime(job_row['run_start_time'])) & (pd.to_datetime(node_measurements['timestamp']) <= pd.to_datetime((job_row['end_time'])) - np.timedelta64(1, 'm'))]
        
        ### TODO drop missing measurements (calculate the percentage)
        tot_intervals = interval_measurements.shape[0]
        interval_measurements = interval_measurements.dropna()
        # print("interval_measurements {}".format(interval_measurements.shape[0]))
        print("{}/{} intervals missing".format(tot_intervals-interval_measurements.shape[0], tot_intervals))
                
        # print(interval_measurements)
        ### drop where active cores is greater than 16 - only on node 30
        interval_measurements = interval_measurements[interval_measurements['active_cores'] <= 16]
        ### TODO drop row with 0 active_cores (or less than the current job used cores)
        interval_measurements = interval_measurements[interval_measurements['active_cores'] >= 1]  
        
        if(interval_measurements.shape[0] != 0):
            good_nodes += 1
        
        ### group the intervals wehere the partial_pow_cons can be obtaied with the same instance of the formula
        ### take the mean for the pow columns
        interval_grouped = interval_measurements.reset_index().groupby(["active_cores", "active_jobs", "active_gpus", "active_mics"]).mean()
        interval_grouped = interval_grouped.loc[:, ~interval_grouped.columns.str.contains('^Unnamed')]
        print(interval_grouped)
        
        if(interval_grouped.shape[0]>1):
            job_runned_alone = False
        
        ### apply the formula on each grouped data
        for index_group, row_group in interval_grouped.iterrows():
            n_active_cores = index_group[0]
            partial_pow_consumption += (row_group['pow_tot_0'] + row_group['pow_tot_1']) * row['ncpus'] / n_active_cores
        
        if(interval_grouped.shape[0] != 0):
            partial_pow_consumption = partial_pow_consumption / interval_grouped.shape[0]
            
        print("partial measurement: {}".format(partial_pow_consumption))
        job_real_pow_consumption += partial_pow_consumption
        partial_pow_consumption = 0
    
    print("{}/{} node measurements are good".format(good_nodes, job_to_nodes_unique.shape[0]))
    
    # print("job_real_pow_consumption is {}".format(job_real_pow_consumption))
    ### TODO if some entire node data is missing, approximate the amount to add
    job_real_pow_consumption = job_real_pow_consumption + (job_real_pow_consumption/good_nodes) * (job_to_nodes_unique.shape[0] - good_nodes)
    
    return job_real_pow_consumption, job_runned_alone

In [70]:
calculate_job_consumption(job_id_string)

job 500752.node129 use 12 cores in this node
job 500752.node129 start at 2014-03-31 18:32:19 and end at 2014-03-31 21:44:58
job 500752.node129 requires 2 nodes and 24 cores
[16 20]
node 16
0/191 intervals missing
                                                  index  pow_tot_0  pow_tot_1
active_cores active_jobs active_gpus active_mics                             
12           1           0           0             1208  79.032066  77.942869
partial measurement: 156.97493494708016
node 20
191/191 intervals missing
Empty DataFrame
Columns: [index, pow_tot_0, pow_tot_1]
Index: []
partial measurement: 0
1/2 node measurements are good


(313.9498698941603, True)

In [33]:
#interval_merged = merged_jobs_to_nodes.loc[(pd.to_datetime(merged_jobs_to_nodes['start_time']) >= start_time) & (pd.to_datetime(merged_jobs_to_nodes['end_time']) <= end_time)]
#interval_merged = interval_merged.sort_values('run_start_time')
#print("interval_merged contains {} records".format(interval_merged.shape[0]))

In [35]:
#interval_merged.head(10)
#jobs = interval_merged.job_id.unique()
#print(jobs.shape[0])

In [80]:
jobs_runned_alone = 0
for job in jobs:
    job_nodes_rows = interval_merged[interval_merged['job_id'] == job]
    print("the job {} runned on {} nodes".format(job, job_nodes_rows.shape[0]))
    job_start_time = job_nodes_rows.run_start_time.iloc[0]
    job_end_time = job_nodes_rows.end_time.iloc[0]
    print("job {} start at {} and end at {}".format(job, str(job_start_time), str(job_end_time)))
    
    job_nodes = job_nodes_rows.node_id.unique()
    print(job_nodes)
    
    if(job_nodes.shape[0] == 1): #if the job is single node
        infile_node = datadir + "CPUs/" + interval_comment + "/node" + str(node) + suffix + interval_comment + "_active_cores_and_jobs.csv"
        node_measurements = pd.read_csv(infile_node)
        
        node_interval_measurements = node_measurements.loc[(node_measurements['timestamp'] >= job_start_time - pd.offsets.Second(60)) & (node_measurements['timestamp'] <= job_end_time)]
        # print(node_interval_measurements)
        print(node_interval_measurements.shape[0])
        if (node_interval_measurements['active_jobs'].sum() / node_interval_measurements.shape[0] == 1):
            # the job runned alone, easy to calculate
            jobs_runned_alone = jobs_runned_alone + 1
            
            
        
    #else:
    #    break
    #    for node in job_nodes:
    #        infile_node = datadir + "CPUs/" + interval_comment + "/node" + str(node) + suffix + interval_comment + "_active_cores_and_jobs.csv"
    #        node_measurements = pd.read_csv(infile_node)
#
#            node_interval_measurements = node_measurements.loc[(node_measurements['timestamp'] >= job_start_time) & (node_measurements['timestamp'] <= job_end_time)]
#            # print(node_interval_measurements)
#            print(node_interval_measurements.shape[0])

the job 1532.0 runned on 1 nodes
job 1532.0 start at 2014-04-01 00:02:39 and end at 2014-04-01 00:32:33
[33]
30
the job 1533.0 runned on 1 nodes
job 1533.0 start at 2014-04-01 00:02:39 and end at 2014-04-01 00:02:50
[33]
0
the job 1534.0 runned on 1 nodes
job 1534.0 start at 2014-04-01 00:02:39 and end at 2014-04-01 00:06:21
[33]
4
the job 1535.0 runned on 1 nodes
job 1535.0 start at 2014-04-01 00:02:39 and end at 2014-04-01 00:32:33
[34]


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


30
the job 1536.0 runned on 1 nodes
job 1536.0 start at 2014-04-01 00:02:40 and end at 2014-04-01 00:02:56
[34]
0
the job 1537.0 runned on 1 nodes
job 1537.0 start at 2014-04-01 00:02:40 and end at 2014-04-01 00:05:03
[34]
3
the job 1538.0 runned on 1 nodes
job 1538.0 start at 2014-04-01 00:02:40 and end at 2014-04-01 00:32:34
[35]
30
the job 1539.0 runned on 1 nodes
job 1539.0 start at 2014-04-01 00:02:40 and end at 2014-04-01 00:02:56
[35]
0
the job 1540.0 runned on 1 nodes
job 1540.0 start at 2014-04-01 00:02:40 and end at 2014-04-01 00:32:34
[36]
30
the job 1541.0 runned on 1 nodes
job 1541.0 start at 2014-04-01 00:02:40 and end at 2014-04-01 00:03:22
[36]
1
the job 1542.0 runned on 1 nodes
job 1542.0 start at 2014-04-01 00:02:41 and end at 2014-04-01 00:32:35
[37]
30
the job 1543.0 runned on 1 nodes
job 1543.0 start at 2014-04-01 00:02:41 and end at 2014-04-01 00:32:35
[38]
30
the job 1544.0 runned on 1 nodes
job 1544.0 start at 2014-04-01 00:02:41 and end at 2014-04-01 00:12:20
[

16
the job 1604.0 runned on 1 nodes
job 1604.0 start at 2014-04-01 00:32:35 and end at 2014-04-01 01:02:28
[36]
30
the job 1603.0 runned on 1 nodes
job 1603.0 start at 2014-04-01 00:32:35 and end at 2014-04-01 01:02:30
[35]
30
the job 1605.0 runned on 1 nodes
job 1605.0 start at 2014-04-01 00:32:35 and end at 2014-04-01 01:02:30
[37]
30
the job 1607.0 runned on 1 nodes
job 1607.0 start at 2014-04-01 00:32:36 and end at 2014-04-01 01:02:30
[40]
30
the job 1608.0 runned on 1 nodes
job 1608.0 start at 2014-04-01 00:32:37 and end at 2014-04-01 01:02:31
[50]
30
the job 1610.0 runned on 1 nodes
job 1610.0 start at 2014-04-01 00:32:38 and end at 2014-04-01 01:02:31
[50]
30
the job 1609.0 runned on 1 nodes
job 1609.0 start at 2014-04-01 00:32:38 and end at 2014-04-01 01:02:32
[50]
30
the job 1611.0 runned on 1 nodes
job 1611.0 start at 2014-04-01 00:32:38 and end at 2014-04-01 01:02:33
[46]
30
the job 1612.0 runned on 1 nodes
job 1612.0 start at 2014-04-01 00:32:38 and end at 2014-04-01 01:02:

14
the job 1686.0 runned on 1 nodes
job 1686.0 start at 2014-04-01 01:03:46 and end at 2014-04-01 01:04:23
[58]
1
the job 1687.0 runned on 1 nodes
job 1687.0 start at 2014-04-01 01:03:49 and end at 2014-04-01 01:04:27
[36]
1
the job 1688.0 runned on 1 nodes
job 1688.0 start at 2014-04-01 01:03:49 and end at 2014-04-01 01:04:27
[59]
1
the job 1689.0 runned on 1 nodes
job 1689.0 start at 2014-04-01 01:03:52 and end at 2014-04-01 01:11:33
[44]
8
the job 1690.0 runned on 1 nodes
job 1690.0 start at 2014-04-01 01:03:54 and end at 2014-04-01 01:04:32
[34]
1
the job 1691.0 runned on 1 nodes
job 1691.0 start at 2014-04-01 01:03:58 and end at 2014-04-01 01:06:06
[64]
3
the job 1692.0 runned on 1 nodes
job 1692.0 start at 2014-04-01 01:03:58 and end at 2014-04-01 01:04:46
[59]
1
the job 1693.0 runned on 1 nodes
job 1693.0 start at 2014-04-01 01:03:58 and end at 2014-04-01 01:04:19
[54]
1
the job 1694.0 runned on 1 nodes
job 1694.0 start at 2014-04-01 01:04:02 and end at 2014-04-01 01:05:13
[35]


...

0
the job 11315.0 runned on 1 nodes
job 11315.0 start at 2014-04-05 04:04:23 and end at 2014-04-05 04:04:26
[57]
0
the job 11316.0 runned on 1 nodes
job 11316.0 start at 2014-04-05 04:04:26 and end at 2014-04-05 04:04:29
[57]
0
the job 11317.0 runned on 1 nodes
job 11317.0 start at 2014-04-05 04:04:29 and end at 2014-04-05 04:04:32
[57]
0
the job 11318.0 runned on 1 nodes
job 11318.0 start at 2014-04-05 04:04:33 and end at 2014-04-05 04:04:35
[57]
0
the job 11319.0 runned on 1 nodes
job 11319.0 start at 2014-04-05 04:04:36 and end at 2014-04-05 04:04:39
[57]
0
the job 11320.0 runned on 1 nodes
job 11320.0 start at 2014-04-05 04:04:39 and end at 2014-04-05 04:04:42
[57]
0
the job 11321.0 runned on 1 nodes
job 11321.0 start at 2014-04-05 04:04:43 and end at 2014-04-05 04:04:45
[57]
0
the job 11322.0 runned on 1 nodes
job 11322.0 start at 2014-04-05 04:04:45 and end at 2014-04-05 04:04:49
[57]
0
the job 11323.0 runned on 1 nodes
job 11323.0 start at 2014-04-05 04:04:49 and end at 2014-04-

0
the job 11389.0 runned on 1 nodes
job 11389.0 start at 2014-04-05 04:08:28 and end at 2014-04-05 04:08:30
[57]
0
the job 11390.0 runned on 1 nodes
job 11390.0 start at 2014-04-05 04:08:31 and end at 2014-04-05 04:08:34
[57]
0
the job 11391.0 runned on 1 nodes
job 11391.0 start at 2014-04-05 04:08:34 and end at 2014-04-05 04:08:37
[57]
0
the job 11392.0 runned on 1 nodes
job 11392.0 start at 2014-04-05 04:08:37 and end at 2014-04-05 04:08:40
[57]
0
the job 11393.0 runned on 1 nodes
job 11393.0 start at 2014-04-05 04:08:41 and end at 2014-04-05 04:08:44
[57]
0
the job 11394.0 runned on 1 nodes
job 11394.0 start at 2014-04-05 04:08:44 and end at 2014-04-05 04:08:47
[57]
0
the job 11395.0 runned on 1 nodes
job 11395.0 start at 2014-04-05 04:08:47 and end at 2014-04-05 04:08:50
[57]
0
the job 11396.0 runned on 1 nodes
job 11396.0 start at 2014-04-05 04:08:51 and end at 2014-04-05 04:08:54
[57]
0
the job 11397.0 runned on 1 nodes
job 11397.0 start at 2014-04-05 04:08:54 and end at 2014-04-

0
the job 11463.0 runned on 1 nodes
job 11463.0 start at 2014-04-05 04:12:48 and end at 2014-04-05 04:12:52
[57]
0
the job 11464.0 runned on 1 nodes
job 11464.0 start at 2014-04-05 04:12:52 and end at 2014-04-05 04:12:55
[57]
0
the job 11465.0 runned on 1 nodes
job 11465.0 start at 2014-04-05 04:12:55 and end at 2014-04-05 04:12:58
[57]
0
the job 11466.0 runned on 1 nodes
job 11466.0 start at 2014-04-05 04:12:59 and end at 2014-04-05 04:13:02
[57]
1
the job 11467.0 runned on 1 nodes
job 11467.0 start at 2014-04-05 04:13:03 and end at 2014-04-05 04:13:05
[57]
0
the job 11468.0 runned on 1 nodes
job 11468.0 start at 2014-04-05 04:13:06 and end at 2014-04-05 04:13:09
[57]
0
the job 11469.0 runned on 1 nodes
job 11469.0 start at 2014-04-05 04:13:09 and end at 2014-04-05 04:13:12
[57]
0
the job 11470.0 runned on 1 nodes
job 11470.0 start at 2014-04-05 04:13:12 and end at 2014-04-05 04:13:15
[57]
0
the job 11471.0 runned on 1 nodes
job 11471.0 start at 2014-04-05 04:13:16 and end at 2014-04-

0
the job 11537.0 runned on 1 nodes
job 11537.0 start at 2014-04-05 04:16:58 and end at 2014-04-05 04:17:01
[57]
1
the job 11538.0 runned on 1 nodes
job 11538.0 start at 2014-04-05 04:17:02 and end at 2014-04-05 04:17:04
[57]
0
the job 11539.0 runned on 1 nodes
job 11539.0 start at 2014-04-05 04:17:05 and end at 2014-04-05 04:17:08
[57]
0
the job 11540.0 runned on 1 nodes
job 11540.0 start at 2014-04-05 04:17:08 and end at 2014-04-05 04:17:11
[57]
0
the job 11541.0 runned on 1 nodes
job 11541.0 start at 2014-04-05 04:17:12 and end at 2014-04-05 04:17:14
[57]
0
the job 11542.0 runned on 1 nodes
job 11542.0 start at 2014-04-05 04:17:15 and end at 2014-04-05 04:17:18
[57]
0
the job 11543.0 runned on 1 nodes
job 11543.0 start at 2014-04-05 04:17:19 and end at 2014-04-05 04:17:21
[57]
0
the job 11544.0 runned on 1 nodes
job 11544.0 start at 2014-04-05 04:17:22 and end at 2014-04-05 04:17:24
[57]


KeyboardInterrupt: 

In [81]:
jobs_runned_alone

672

In [82]:
# test pd.offsets
pd.to_datetime('2014-04-01 00:02:39') - pd.offsets.Second(60)

Timestamp('2014-04-01 00:01:39')

In [69]:
completed_jobs_data_interval[completed_jobs_data_interval['job_id'] == 1532]

,job_id,job_id_string,start_time,run_start_time,end_time,node_req,cpu_req,mem_req,time_req,exit_status,entire_node,real_power
1531,1532,501704.node129,2014-04-01 00:02:38,2014-04-01 00:02:39,2014-04-01 00:32:33,1,1,4,00:30,COMPLETED,NaN,NaN
